In [7]:
import tensorflow as tf

img_size = (128, 128)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/train",
    image_size=(128, 128),
    batch_size=32,
    label_mode="categorical"
).apply(tf.data.experimental.ignore_errors())

val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/validate",
    image_size=(128, 128),
    batch_size=32,
    label_mode="categorical"
).apply(tf.data.experimental.ignore_errors())

test_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/test",
    image_size=(128, 128),
    batch_size=32,
    label_mode="categorical"
).apply(tf.data.experimental.ignore_errors())

Found 17581 files belonging to 30 classes.
Found 2515 files belonging to 30 classes.
Found 5040 files belonging to 30 classes.


In [8]:
# Normalize images and add augmentation for training
AUTOTUNE = tf.data.AUTOTUNE

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

def preprocess(ds, augment=False):
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y), num_parallel_calls=AUTOTUNE)
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE)
    return ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

train_ds = preprocess(train_ds, augment=True)
val_ds = preprocess(val_ds)
test_ds = preprocess(test_ds)

In [9]:
from tensorflow.keras import layers, models

num_classes = 30

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 30)             │         7,710 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,523,742 (24.89 MB)

 Trainable params: 6,523,742 (24.89 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)

Epoch 1/20


2025-05-14 01:04:04.816147: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2025-05-14 01:04:05.156791: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
Corrupt JPEG data: 229 extraneous bytes before marker 0xd9
2025-05-14 01:04:10.983977: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2025-05-14 01:04:13.519008: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:77: Filling up shuffle buffer (this may take a while): 205 of 1000
2025-05-14 01:04:22.033475: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2025-05-14 01:04:22.033508: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: cHRM chunk does not match sRGB
Corrupt JPEG data: 9 extraneous bytes before marker 0xe2
2025-05-14 01:04:23.579541: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:77: Filling up shuffle buffer

    549/Unknown 92s 111ms/step - accuracy: 0.0676 - loss: 3.3767

2025-05-14 01:05:35.004774: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-05-14 01:05:35.007235: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10771841404878800312
2025-05-14 01:05:35.009322: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/sequential_3_1/dropout_1_1/Cast/_69]]
2025-05-14 01:05:35.009333: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2106600322941076934
2025-05-14 01:05:35.009339: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 1896224269100630053
2025-05-14 01:05:35.009341: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled.

549/549 ━━━━━━━━━━━━━━━━━━━━ 101s 127ms/step - accuracy: 0.0677 - loss: 3.3765 - val_accuracy: 0.1392 - val_loss: 3.0896
Epoch 2/20
  1/549 ━━━━━━━━━━━━━━━━━━━━ 1:30 166ms/step - accuracy: 0.0938 - loss: 3.4024

...

2025-05-14 01:18:07.352002: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-05-14 01:18:07.352015: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/BitwiseAnd/_8]]
2025-05-14 01:18:07.352022: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6103345246978917777
2025-05-14 01:18:07.352026: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9843803076241735003
2025-05-14 01:18:07.352030: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11603336784602908087
2025-05-14 01:18:07.352035: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16093764271

549/549 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.2799 - loss: 2502.0259

2025-05-14 01:18:45.726887: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-05-14 01:18:45.726914: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11331704232733085716
2025-05-14 01:18:45.726931: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/adam/Add_8/_48]]
2025-05-14 01:18:45.726946: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8112348920461893821
2025-05-14 01:18:45.726951: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16300645003003012307
2025-05-14 01:18:45.726954: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 392735863

549/549 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.2798 - loss: 2502.4702 - val_accuracy: 0.2235 - val_loss: 2865.1711


2025-05-14 01:18:48.656986: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-05-14 01:18:48.657002: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16093764271703836826
2025-05-14 01:18:48.657008: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_10]]
2025-05-14 01:18:48.657021: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9843803076241735003
2025-05-14 01:18:48.657027: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2873146264412591386
2025-05-14 01:18:48.657034: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9044742654603874156
2025-05-1

In [11]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.2f}")


2025-05-14 01:18:50.847519: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile


158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.2347 - loss: 2914.2119
Test accuracy: 0.23


2025-05-14 01:18:58.816374: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-05-14 01:18:58.816404: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16093764271703836826
2025-05-14 01:18:58.816430: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2873146264412591386
2025-05-14 01:18:58.816438: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9044742654603874156
2025-05-14 01:18:58.816443: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_10]]
2025-05-14 01:18:58.816459: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9843803076241735003
2025-05-1

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)

Epoch 1/20


2025-05-14 01:04:04.816147: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2025-05-14 01:04:05.156791: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
Corrupt JPEG data: 229 extraneous bytes before marker 0xd9
2025-05-14 01:04:10.983977: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2025-05-14 01:04:13.519008: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:77: Filling up shuffle buffer (this may take a while): 205 of 1000
2025-05-14 01:04:22.033475: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2025-05-14 01:04:22.033508: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: cHRM chunk does not match sRGB
Corrupt JPEG data: 9 extraneous bytes before marker 0xe2
2025-05-14 01:04:23.579541: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:77: Filling up shuffle buffer

    549/Unknown 92s 111ms/step - accuracy: 0.0676 - loss: 3.3767

2025-05-14 01:05:35.004774: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-05-14 01:05:35.007235: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10771841404878800312
2025-05-14 01:05:35.009322: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/sequential_3_1/dropout_1_1/Cast/_69]]
2025-05-14 01:05:35.009333: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2106600322941076934
2025-05-14 01:05:35.009339: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 1896224269100630053
2025-05-14 01:05:35.009341: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled.

549/549 ━━━━━━━━━━━━━━━━━━━━ 101s 127ms/step - accuracy: 0.0677 - loss: 3.3765 - val_accuracy: 0.1392 - val_loss: 3.0896
Epoch 2/20
  1/549 ━━━━━━━━━━━━━━━━━━━━ 1:30 166ms/step - accuracy: 0.0938 - loss: 3.4024

...

2025-05-14 01:18:07.352002: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-05-14 01:18:07.352015: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/BitwiseAnd/_8]]
2025-05-14 01:18:07.352022: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6103345246978917777
2025-05-14 01:18:07.352026: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9843803076241735003
2025-05-14 01:18:07.352030: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11603336784602908087
2025-05-14 01:18:07.352035: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16093764271

549/549 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.2799 - loss: 2502.0259

2025-05-14 01:18:45.726887: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-05-14 01:18:45.726914: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11331704232733085716
2025-05-14 01:18:45.726931: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/adam/Add_8/_48]]
2025-05-14 01:18:45.726946: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8112348920461893821
2025-05-14 01:18:45.726951: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16300645003003012307
2025-05-14 01:18:45.726954: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 392735863

549/549 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.2798 - loss: 2502.4702 - val_accuracy: 0.2235 - val_loss: 2865.1711


2025-05-14 01:18:48.656986: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-05-14 01:18:48.657002: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16093764271703836826
2025-05-14 01:18:48.657008: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_10]]
2025-05-14 01:18:48.657021: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9843803076241735003
2025-05-14 01:18:48.657027: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2873146264412591386
2025-05-14 01:18:48.657034: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9044742654603874156
2025-05-1